# Pandas

Pandas is a powerful library for data manipulation and analysis in Python. It is widely used in a range of fields |  including data science |  finance |  and statistics.

In [112]:
from pathlib import Path
from datetime import datetime
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

pd.set_option('display.max_rows', None)
InteractiveShell.ast_node_interactivity = "all"

## 009. Linear regression in a SQLite DB

Implementation of this GitHub ticket https://github.com/gotofritz/yarkie/issues/10

## 009.000 Assets and globals

009.sqlit.db:

| rowid | ranking | id          | view_count | comment_count | like_count | upload_date | comment_count_estimated |
|-------|---------|-------------|-----------|----------------|------------|-------------|-------------------------|
| 505   | 36      | X-uJtV8ScYk | 334895639  | 2700000       | 5985543    | 20200914    | 0                       |
...


In [113]:

DB = Path("009_sqlite.db")
TARGET_COLUMN = "comment_count_estimated"

## 009.001 Read data

1. Read columns id, view_count, like_count, upload_date, comment_count in table videos from DB into a dataframe
    1. No need for an index
    1. Print the head
1. Massage the data
    1. `upload_date` is a string but not always in constant format. Use pd builtin function to turn into a consistent datetime
    1. Create a new column `days_since_upload` which is the difference between today and `upload_date`
    1. Create three columns, `v`, `l`, `c` which are the three `xxxx_count` divided by `days_since_upload`
    1. copy `comment_count` to TARGET_COLUMN
    1. Print the head, sorting by index
1. Create `df_missing_data` and `df_non_missing_data` with rows with `comment_count` = 0  or > 0 respectively
    1. Print the head for both, sorting by index

In [114]:
import sqlite3 as sqlite

# solution
1, "SQL"
con = sqlite.connect(DB)
df = pd.read_sql(f"select id, view_count, like_count, upload_date, comment_count, {TARGET_COLUMN} from videos", con=con)
df.head()

2, "MASSAGED"
df["upload_date"] = pd.to_datetime(df['upload_date'], format='mixed')
df["days_since_upload"] = (datetime.today() - df["upload_date"]).dt.days
df["v"] = df["view_count"] / df["days_since_upload"]
df["l"] = df["like_count"] / df["days_since_upload"]
df["c"] = df["comment_count"] / df["days_since_upload"]
df[TARGET_COLUMN] = df["comment_count"]
df.sort_index().head()

3, "MISSING / NON MISSING"
df_missing_data = df[df[TARGET_COLUMN] == 0]
df_missing_data.sort_index().head()
df_non_missing_data = df[df[TARGET_COLUMN] != 0]
df_non_missing_data.sort_index().head()

(1, 'SQL')

,id,view_count,like_count,upload_date,comment_count,comment_count_estimated
0,Tis5Tm7PAwM,7831,800,20230913,95,0
1,PGDSawOwHkw,597,22,20230131,0,0
2,4ID2bOYTj94,774,13,20230131,0,0
3,3f1lgHaldYY,681,9,20230131,0,0
4,kbax_kx380o,422,8,20230131,0,0


(2, 'MASSAGED')

,id,view_count,like_count,upload_date,comment_count,comment_count_estimated,days_since_upload,v,l,c
0,Tis5Tm7PAwM,7831,800,2023-09-13,95,95,60,130.516667,13.333333,1.583333
1,PGDSawOwHkw,597,22,2023-01-31,0,0,285,2.094737,0.077193,0.000000
2,4ID2bOYTj94,774,13,2023-01-31,0,0,285,2.715789,0.045614,0.000000
3,3f1lgHaldYY,681,9,2023-01-31,0,0,285,2.389474,0.031579,0.000000
4,kbax_kx380o,422,8,2023-01-31,0,0,285,1.480702,0.028070,0.000000


(3, 'MISSING / NON MISSING')

,id,view_count,like_count,upload_date,comment_count,comment_count_estimated,days_since_upload,v,l,c
1,PGDSawOwHkw,597,22,2023-01-31,0,0,285,2.094737,0.077193,0.0
2,4ID2bOYTj94,774,13,2023-01-31,0,0,285,2.715789,0.045614,0.0
3,3f1lgHaldYY,681,9,2023-01-31,0,0,285,2.389474,0.031579,0.0
4,kbax_kx380o,422,8,2023-01-31,0,0,285,1.480702,0.028070,0.0
5,QWqsz25NpsU,360,6,2023-01-31,0,0,285,1.263158,0.021053,0.0


,id,view_count,like_count,upload_date,comment_count,comment_count_estimated,days_since_upload,v,l,c
0,Tis5Tm7PAwM,7831,800,2023-09-13,95,95,60,130.516667,13.333333,1.583333
6,YUX8fUrKRNU,739128,7515,2023-01-30,278,278,286,2584.363636,26.276224,0.972028
7,tuqs6clFBRE,23536,329,2022-12-07,16,16,340,69.223529,0.967647,0.047059
8,ukJXPUODzcM,186808,2750,2022-10-25,77,77,383,487.749347,7.180157,0.201044
9,gVPUAntzOl4,932226,13527,2022-10-03,367,367,405,2301.792593,33.400000,0.906173


## 009.003 Guestimate the data

1. Import the scikit-learn library for linear regression
1. Create a model
    1. Use the v, c, l columns to train it
    1. Use the model to estimate the missing data
1. Overwrite the TARGET_COLUMN with the guessed values by multiplying `c` by `days_since_upload`
    1. Print the head, sorting by index

In [115]:
# solution

1
from sklearn.linear_model import LinearRegression

2
regression_model = LinearRegression()
regression_model.fit(
    df_non_missing_data[['v', 'l']],
    df_non_missing_data['c']
)
df_missing_data.loc[:, 'c'] = regression_model.predict(
    df_missing_data[['v', 'l']]
)

3
df_missing_data.loc[:, TARGET_COLUMN] = (
    df_missing_data["c"] * df_missing_data["days_since_upload"]
).round().astype(int)
df_missing_data.sort_index().head()



1

2

LinearRegression()

3

,id,view_count,like_count,upload_date,comment_count,comment_count_estimated,days_since_upload,v,l,c
1,PGDSawOwHkw,597,22,2023-01-31,0,83,285,2.094737,0.077193,0.292748
2,4ID2bOYTj94,774,13,2023-01-31,0,84,285,2.715789,0.045614,0.293011
3,3f1lgHaldYY,681,9,2023-01-31,0,84,285,2.389474,0.031579,0.293013
4,kbax_kx380o,422,8,2023-01-31,0,83,285,1.480702,0.028070,0.292857
5,QWqsz25NpsU,360,6,2023-01-31,0,83,285,1.263158,0.021053,0.292848


## 009.004 Update the DB

1. Create a list of records in the format `{'id': 'PGDSawOwHkw', '{TARGET_COLUMN}': 438}, ...` 
    1. Start from `df_missing_data`
    1. Add `df_non_missing_data`
    1. Print the first 5 records
    1. Print the 
1. Write the data into the db
    1. Initialise the DB in sqlite_utils
    1. assert that the number of records with are going to update is the same is the count of all records in that table
    1. Use `upsert_all` to update records (there is no `update_add`). Don't forget the pk

In [116]:
from sqlite_utils import Database

# solution

1
records = df_missing_data[["id", TARGET_COLUMN]].to_dict(orient="records")
records.extend(
    df_non_missing_data[["id", TARGET_COLUMN]].to_dict(orient="records")
)
records[:5]

2
db = Database(DB)
assert len(records) == db["videos"].count
db["videos"].upsert_all(records=records, pk="id")




1

[{'id': 'PGDSawOwHkw', 'comment_count_estimated': 83},
 {'id': '4ID2bOYTj94', 'comment_count_estimated': 84},
 {'id': '3f1lgHaldYY', 'comment_count_estimated': 84},
 {'id': 'kbax_kx380o', 'comment_count_estimated': 83},
 {'id': 'QWqsz25NpsU', 'comment_count_estimated': 83}]

2

<Table videos (id, ranking, view_count, comment_count, like_count, upload_date, comment_count_estimated)>